In [2]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

import fasttext
from sentence_transformers import SentenceTransformer
from gensim.models.fasttext import load_facebook_model

import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm
import utils

# reload import utils
from importlib import reload
reload(utils)


<module 'utils' from '/home/data/hussam/sparta/youtube-sim/src/utils.py'>

In [35]:
df = pd.read_parquet('../data/logs.parquet')
meta_data = pd.read_pickle('../data/videos_raw_metadata')
embeddings = pd.read_pickle('../data/embeddings/openai-title')

In [71]:
videos = utils.clean(meta_data, titles=embeddings)

In [52]:
df = df[df['is_video']]
data = dict()
users = list(set(df['userId']))
for user in tqdm(users):
    data[user] = df[df['userId'] == user].sort_values('date')

100%|██████████| 1863/1863 [00:35<00:00, 52.77it/s]


In [72]:
watches = dict()
for user in tqdm(users):
    watches[user] = []
    user_data = data[user][['date', 'url', 'is_video', 'upnext_content']].to_dict('records')
    for wd in user_data:
        watch = dict()
        watch['playing'] = videos.get(utils.get_id(wd['url']), None)
        if wd['upnext_content'] is not None:
            watch['suggested'] = [videos.get(u, None) for u in utils.decode_upnext(wd['upnext_content'])]
        else:
            watch['suggested'] = []
        watches[user].append(watch)

100%|██████████| 1863/1863 [00:25<00:00, 73.81it/s] 


In [164]:
for user in watches:
    if len(watches[user]) == 1:
        watches[user][0]['selected'] = None
        continue
    w = 0
    for w in range(len(watches[user]) - 1):
        nw = w + 1
        if watches[user][w]['playing'] is None or watches[user][nw]['playing'] is None or watches[user][w]['suggested'] == []:
            watches[user][w]['selected'] = None
            continue

        if watches[user][w]['suggested'] != None:
            suggested = []
            for s in watches[user][w]['suggested']:
                if s is not None:
                    suggested.append(s.get('id', None))
        else:
            watches[user][w]['selected'] = None
            continue

        if watches[user][nw]['playing']['id'] in suggested:
            watches[user][w]['selected'] = watches[user][nw]['playing']['id']
        else:
            watches[user][w]['selected'] = None

    watches[user][nw]['selected'] = None


In [170]:
pkl.dump(watches, open('../data/user-level-data', 'wb'))

In [4]:
watches = pd.read_pickle('../data/user-level-data')